In [ ]:
from config import DEEPSEEK_API_KEY, OPENROUTER_API_KEY
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,#Your model API key here
)

messages=[{"role": "user", "content": """
You will write python program to solve the below problem. You will only write code
blocks. Your python promgram must be executable and returns the right answer for the
problem.
Q:You are a helpful assistant. Solve this puzzle for me.
In this puzzle, there are stacks of blocks, and the goal is to rearrange them into a target
configuration using a sequence of moves where:
• Only the topmost block from any stack can be moved.
• A block can be placed either on an empty position or on top of another block.
Example: With initial state [["A", "B"], ["C"], []] and goal state [["A"], ["B"],
["C"]], a solution might be:
moves = [[" C " , 1 , 2] , [" B " , 0 , 1]]
This means: Move block C from stack 1 to stack 2, then move block B from stack 0 to stack 1.
Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• Ensure your final answer also includes the complete list of moves for final solution in the
format: moves = [[block, from stack, to stack], ...]
I have a puzzle with $N$ blocks.
Initial state:
Stack 0: $blocks_0$ (top)
Stack 1: $blocks_1$ (top)
...
Stack $m$: $blocks_m$ (top)
Goal state:
Stack 0: $goal_blocks_0$ (top)
Stack 1: $goal_blocks_1$ (top)
...
Stack $m$: $goal_blocks_m$ (top)
Find the minimum sequence of moves to transform the initial state into the goal state. Remember
that only the topmost block of each stack can be moved.


# solution using Python:
def solution():
"You are a helpful assistant. Solve this puzzle for me.
In this puzzle, there are stacks of blocks, and the goal is to rearrange them into a target
configuration using a sequence of moves where:
• Only the topmost block from any stack can be moved.
• A block can be placed either on an empty position or on top of another block.
Example: With initial state [["A", "B"], ["C"], []] and goal state [["A"], ["B"],
["C"]], a solution might be:
moves = [[" C " , 1 , 2] , [" B " , 0 , 1]]
This means: Move block C from stack 1 to stack 2, then move block B from stack 0 to stack 1.
Requirements:
• When exploring potential solutions in your thinking process, always include the corresponding complete list of moves.
• Ensure your final answer also includes the complete list of moves for final solution in the
format: moves = [[block, from stack, to stack], ...]
I have a puzzle with $N$ blocks.
Initial state:
Stack 0: $blocks_0$ (top)
Stack 1: $blocks_1$ (top)
...
Stack $m$: $blocks_m$ (top)
Goal state:
Stack 0: $goal_blocks_0$ (top)
Stack 1: $goal_blocks_1$ (top)
...
Stack $m$: $goal_blocks_m$ (top)
Find the minimum sequence of moves to transform the initial state into the goal state. Remember
that only the topmost block of each stack can be moved."

"""}]

completion = client.chat.completions.create(
  extra_headers={
  },
  extra_body={"enable_thinking": True},
  model="qwen/qwen3-32b",
  max_tokens=64000,
  messages=messages
)
print(completion.choices[0].message.content)

To solve this block-moving puzzle, we employ **Breadth-First Search (BFS)**. BFS is ideal for finding the **shortest sequence of moves** that transforms the initial configuration of stacks into the goal configuration. Each move is defined as transferring the **top block** of one stack to **any other allowed location**.

---

### ✅ Key Constraints

1. Only the **top block** of a stack can be moved.
2. Blocks can be placed either on an **empty stack** or on top of **another block**.
3. A block cannot be moved from a stack to itself.
4. Solution must return the **minimum number of moves** in a list of the form:  
   `[[block, from_stack, to_stack], ...]`

---

### 🔁 BFS Strategy Overview

- **State Representation**: Each state is represented as a `tuple of tuples`, where each inner tuple represents a stack.
- **Visited States**: To avoid revisiting the same state, we use a `set`.
- **Queue Structure**: The BFS queue contains tuples of the form `(current_state, moves_so_far)`.
- **Goal Che

In [ ]:
from collections import deque

def solution():
    def bfs(initial, goal):
        initial_state = tuple(tuple(stack) for stack in initial)
        goal_state = tuple(tuple(stack) for stack in goal)

        queue = deque()
        queue.append((initial_state, []))

        visited = set()
        visited.add(initial_state)

        while queue:
            current, moves = queue.popleft()

            if current == goal_state:
                return moves

            num_stacks = len(current)
            for source in range(num_stacks):
                if len(current[source]) == 0:
                    continue
                block = current[source][-1]
                for target in range(num_stacks):
                    if source == target:
                        continue
                    new_state = [list(stack) for stack in current]
                    moved_block = new_state[source].pop()
                    new_state[target].append(moved_block)
                    new_state_tuple = tuple(tuple(stack) for stack in new_state)
                    if new_state_tuple not in visited:
                        visited.add(new_state_tuple)
                        new_moves = moves + [[block, source, target]]
                        queue.append((new_state_tuple, new_moves))
        return None

    # Replace the initial_state and goal_state with your puzzle's data
    initial_state = [["A", "B" ,"C"], [], []]
    goal_state = [["A"], ["B"], ["C"]]

    return bfs(initial_state, goal_state)
print(solution())

[['C', 0, 2], ['B', 0, 1]]


In [ ]:
from typing import List, Tuple

State = List[List[str]]
Move  = Tuple[str, int, int]      # (block, src_stack, dst_stack)

def validate_moves(moves: List[Move],
                   initial: State,
                   goal: State,
                   verbose: bool = False) -> bool:


    stacks = [col[:] for col in initial]

    def top(idx: int):
        return stacks[idx][-1] if stacks[idx] else None

    for step, (blk, src, dst) in enumerate(moves, start=1):

        if not stacks[src]:
            raise ValueError(f"Step {step}: source stack {src} is empty.")
        if top(src) != blk:
            raise ValueError(
                f"Step {step}: top of stack {src} is {top(src)}, expected {blk}.")


        stacks[src].pop()
        stacks[dst].append(blk)

        if verbose:
            print(f"After step {step}: {stacks}")


    if stacks != goal:
        raise AssertionError(
            f"Final state differs from goal.\nGot : {stacks}\nGoal: {goal}")

    if verbose:
        print("✓ All moves valid and goal reached!")
    return True

if __name__ == "__main__":

    moves = []


    initial_state = []
    goal_state = []


    validate_moves(moves, initial_state, goal_state, verbose=True)

After step 1: [['A', 'B', 'C', 'D'], [], ['E']]
After step 2: [['A', 'B', 'C'], [], ['E', 'D']]
After step 3: [['A', 'B'], ['C'], ['E', 'D']]
After step 4: [['A', 'B'], ['C', 'D'], ['E']]
✓ All moves valid and goal reached!
